In [37]:
%load_ext autoreload

%autoreload 2

import gauleg as gl 
import numpy as np 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
# initial variables 

a = 0 
b = 1 
l = 2 # for a 4 slot problem 
h = 2 **(-l) # distance between xi, xi+1 is 1 * 2**-l 
n1 = 5 # number of nodes 

n2 = 5 # number of roots in each GL 
ijlist = [1,2,3]

d_phi_i = [0, 1/(2**-l) , (-1/(2**-l))] # only 2 possibilities for d_phi_i 
d_phi_j = [0, 1/(2**-l) , (-1/(2**-l))]
dim = 2**l -1 # number of phi functions i have for l 

### sanity check ###
"""
if l = 2 i expect to have 4 slots and 3 functions and 5 points 
2**l = 2**2 = 4 --> [a = x0,x1,x2,x3,x4 = b]
dim of matrix = 3 = 2**2 -1 
gradient = 1 / 2**-l 
n = 5  --> number of points 
"""


x_list = gl.listi(a,b,h,n)
print(x_list)


a = gl.a(0.25)
print(a)

matrix = np.zeros((dim, dim))
print(matrix)






[0.0, 0.25, 0.5, 0.75, 1.0]
2.2474039592545227
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [75]:
results = gl.gauleg(n2 = 5)
print(results)
print(results[0])


([-0.906179845938664, -0.5384693101056831, 0.0, 0.5384693101056831, 0.906179845938664], [0.236926885056189, 0.47862867049936647, 0.5688888888888889, 0.47862867049936647, 0.236926885056189])
[-0.906179845938664, -0.5384693101056831, 0.0, 0.5384693101056831, 0.906179845938664]


In [77]:
gl.GL(xi = results[0], ci = results[1], y = 0.25, x= 0, n2 = 5 )


0.5310875782893552